### <font color='#00d2d3'> Importer les packages

In [305]:
# Importer les packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords') # Télécharger le package stopwords
nltk.download('wordnet')
from nltk.corpus import stopwords # Importer le package stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### <font color='#00d2d3'> Importer et lire les données

In [306]:
# Importer les données
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [307]:
# Afficher les données
data=pd.read_csv('/content/drive/MyDrive/Future_Intern/reddit_data.csv')
data

,title,selftext,created_utc,author,subreddit
0,Is this considered domestic violence?,\nMe (28f) nd my boyfriend (34m) have been in ...,1.720186e+09,Double-Back-777,relationships
1,My family expects me (30F) to leave my family ...,My mother is disabled due to a neurological il...,1.561784e+09,LittleMissPotatoe,relationships
2,My [22f] boyfriend [23m] got arrested on a dom...,"My boyfriend, Trevor and I live with a family ...",1.471977e+09,Upandaway3435,relationships
3,[UPDATE] Me [28 F] with my BF [29 M] of 3 year...,Original post: https://www.reddit.com/r/relati...,1.443372e+09,annoyedthrw,relationships
4,Last night my (25F) boyfriend of 6 months (30M...,"Basically the title, but I will elaborate a li...",1.690806e+09,Stock-Property1326,relationships
...,...,...,...,...,...
13242,What's the worst relationship advice you've ev...,I was told to divorce my husband for being pou...,1.445544e+09,anxiousgirl89,AskWomen
13243,Do you believe that love is the most important...,My sister's boyfriend treats her like shit. He...,1.420999e+09,UFOtookmysheep,AskWomen
13244,Why do so many women love stories such as Twil...,I find it perplexing that so many women enjoy ...,1.413976e+09,Bendersass,AskWomen
13245,What are some small things that you silently j...,NaN,1.544510e+09,MrRomantico,AskWomen


- 1 : Negative
- 0 : Positive

In [308]:
# Dimension des données
data.shape

(13247, 5)

### <font color='#00d2d3'> Pré-traitement des données

In [309]:
print(data['selftext'].apply(type).value_counts())


selftext
<class 'str'>      12375
<class 'float'>      872
Name: count, dtype: int64


#### <font color='#00d2d3'> Pré-traitement des données textuelles

In [311]:
# Afficher la première ligne de la colonne text
data['title'][0]

'Is this considered domestic violence?'

In [312]:
# Afficher la deuxième ligne de la colonne text
data['selftext'][0]

'\nMe (28f) nd my boyfriend (34m) have been in a relationship for 2 years. We got into an argument last night. It started over this girl he used to work with as I’d seen he had liked one of her photos and I asked if they still talked. He said no look at my phone if you want. So I called his bluff and I was like fine I will. When I went to check their chat was empty and I know they had previously messaged. I asked him why he deleted the chat and he said he did it ages ago to stop our arguments. He tried to grab his phone back but I was clinging onto it and kept saying but why are they deleted. He eventually tackled the phone off of me and left a mark on my arm and broke 2 of my nails. Is this domestic abuse? I have a very dark feeling that my partner just hit me but he insists it was just because we tackled for the phone and I wouldn’t let go and that he hadn’t “hit or abused me”. I’m very confused and upset, the deleted messages are a separate issue to this. I’d like clarification on t

###### <font color='#1dd1a1'> Commencer le nettoyage des tweets


In [313]:
# Fonction supprimant les links
def remove_links(title):
    title= re.sub(r'http:?//\S+ | https:?//\S+','',title)
    return title

In [314]:
def remove_links2(selftext):
    if isinstance(selftext, str):
        return re.sub(r'http:?//\S+ | https:?//\S+','',selftext)
    return selftext

In [315]:
# Fonction supprimant les @username
def remove_users(title):
    title = re.sub(r'@[\w\-._]+','',title)
    return title

In [316]:
def remove_users2(selftext):
    if isinstance(selftext, str):
        # Supprime les mentions utilisateurs (exemple : @username)
        return re.sub(r"@\w+", "", selftext)
    return selftext

In [317]:
# Supprimer les adresses emails
def email_address(text):
  title= re.sub(r'@[\w\-._]+','',text)
  return title

In [318]:
# Supprimer les adresses emails
def email_address2(selftext):
    if isinstance(selftext, str):  # Vérifiez si l'entrée est une chaîne de caractères
        # Supprime les adresses email
        return re.sub(r'@[\w\-._]+', '', selftext)
    return selftext

In [319]:
# Exemple
tweet ='My email adress is the following : abdoulwahabdiall@gmail.com'
tweet = email_address(tweet)

In [320]:
!pip install contractions

In [321]:
import contractions
# Fonction étendant les contractions
def contraction(title):
    expanded_all = []
    for word in title.split():
        expanded_all.append(contractions.fix(word)) # utiliser la fonction fix de contractions

    expand = ' '.join(expanded_all)
    return expand

In [322]:
def contraction2(selftext):
    if isinstance(selftext, str):  # Vérifiez si l'entrée est une chaîne
        expanded_all = [contractions.fix(word) for word in selftext.split()]
        return ' '.join(expanded_all)
    return selftext

In [323]:
# Exemple
tweet ="That's the right thing to do"
tweet = contraction(tweet)
tweet

'That is the right thing to do'

In [324]:
# Supprimer html caractères
def clean_html(text):
   title = re.sub(r'&\w+','',text)
   return title

In [325]:
def clean_html2(selftext):
    if isinstance(selftext, str):  # Vérifiez si l'entrée est une chaîne
        return re.sub(r'&\w+;', '', selftext)
    return selftext

In [326]:
# Exemple
tweet ='#world What a shame &amp'
tweet = clean_html(tweet)
tweet

'#world What a shame '

In [327]:
# Remplacer tout ce qui n'est chaines de caractères alphabétiques et espace par ' '
def alpha_b(text):
   title = re.sub(r'^a-zA-Z\s]+',' ',text)
   return title

In [328]:
def alpha_b2(selftext):
    if isinstance(selftext, str):  # Vérifiez si l'entrée est une chaîne
        # Supprimer tout sauf les lettres (a-z, A-Z) et les espaces
        return re.sub(r'[^a-zA-Z\s]+', ' ', selftext)
    return selftext

In [329]:
# Exemple
tweet = 'My year of birth is 2050 #happy'
tweet = alpha_b(tweet)
tweet

'My year of birth is 2050 #happy'

In [330]:
# Fonction remplaçant les espaces multiples et convertissant majuscules en minuscules
def lower(text):
    title = re.sub(r'\s(2,)',' ' ,text)
    return title

In [331]:
def lower2(selftext):
    if isinstance(selftext, str):  # Vérifiez si l'entrée est une chaîne
        return selftext.lower()
    return selftext

In [332]:
# Supprimer les espaces en début et fin de tweet
def clean_space(text):
    title = re.sub(r'^\s|\s$',' ' ,text)
    return title

In [333]:
def clean_space2(selftext):
    if isinstance(selftext, str):  # Vérifiez si l'entrée est une chaîne
        # Supprimer les espaces en début (^\\s) et en fin (\\s$) de chaîne
        return re.sub(r'^\s+|\s+$', '', selftext)
    return selftext

In [334]:
# Fonction supprimant les stopwords
def remove_stopwords(text):
    Stopwords = stopwords.words('english')
    title= ' '.join([word for word in text.split() if word not in Stopwords])
    return title

In [335]:
def remove_stopwords2(selftext):
    # Charger la liste des mots vides (stopwords)
    Stopwords = set(stopwords.words('english'))

    # Vérifier si l'entrée est une chaîne
    if isinstance(selftext, str):
        # Supprimer les mots vides
        return ' '.join([word for word in selftext.split() if word.lower() not in Stopwords])
    return selftext

In [336]:
# Exemple
tweet = 'Life is a mess for some people'
tweet = remove_stopwords(tweet)
tweet

'Life mess people'

In [337]:
# Lemmatization
from nltk.stem import WordNetLemmatizer
lemma=WordNetLemmatizer()
def lem_sw(text):
    title = [lemma.lemmatize(word) for word in text.split()]
    title = " ".join(title)
    return title

In [338]:
def lem_sw2(selftext):
    if isinstance(selftext, str):  # Vérifiez si l'entrée est une chaîne
        # Appliquer la lemmatisation sur chaque mot
        lemmatized_words = [lemma.lemmatize(word) for word in selftext.split()]
        # Joindre les mots lemmatisés en une chaîne
        return " ".join(lemmatized_words)
    return selftext

###### <font color='#1dd1a1'> Appliquer les différentes fonctions sur letitle



In [339]:
# Appliquer la fonction remove_users
data['new_title'] = data.title.apply(func =remove_users)
# Appliquer la fonction remove_links
data['new_title'] = data.new_title.apply(func =remove_links)
# Appliquer la fonction email_address
data['new_title'] = data.new_title.apply(func =email_address)
# Appliquer la fonction remove_contraction
data['new_title'] = data.new_title.apply(func = contraction)
# Appliquer la fonction clean_html
data['new_title'] = data.new_title.apply(func =clean_html)
# Appliquer la fonction alpha_b
data['new_title'] = data.new_title.apply(func =alpha_b)
# Appliquer la fonction lower
data['new_title'] = data.new_title.apply(func =lower)
# Appliquer la fonction clean_space
data['new_title'] = data.new_title.apply(func =clean_space)
# Appliquer la fonction remove_stopwords
data['new_title'] = data.new_title.apply(func =remove_stopwords)
# Appliquer la fonction lem_sw
data['new_title'] = data.new_title.apply(func =lem_sw)

In [341]:
# Appliquer la fonction remove_users
data['new_selftext'] = data.selftext.apply(func =remove_users2)
# Appliquer la fonction remove_links
data['new_selftext'] = data.new_selftext.apply(func =remove_links2)
# Appliquer la fonction email_address
data['new_selftext'] = data.new_selftext.apply(func =email_address2)
# Appliquer la fonction remove_contraction
data['new_selftext'] = data.new_selftext.apply(func = contraction2)
# Appliquer la fonction clean_html
data['new_selftext'] = data.new_selftext.apply(func =clean_html2)
# Appliquer la fonction alpha_b
data['new_selftext'] = data.new_selftext.apply(func =alpha_b2)
# Appliquer la fonction lower
data['new_selftext'] = data.new_selftext.apply(func =lower2)
# Appliquer la fonction clean_space
data['new_selftext'] = data.new_selftext.apply(func =clean_space2)
# Appliquer la fonction remove_stopwords
data['new_selftext'] = data.new_selftext.apply(func =remove_stopwords2)
# Appliquer la fonction lem_sw
data['new_selftext'] = data.new_selftext.apply(func =lem_sw2)

In [342]:
# Afficher la ligne d'index 0
data['new_title'][0]

'Is considered domestic violence?'

In [343]:
# Afficher la ligne d'index 1
data['new_selftext'][1]

'mother disabled due neurological illness diagnosed late thirty remember mostly sick needing help used really look super mom could overcome obstacle father died father passed decade ago still high school brother lived parent time got job high school passed cleaned house mowed grass bought food money job paid bill could minimum wage much made dinner night mom brother rarely home unless sleep wanted money year brother prison dozen dozen time everything drunk driving drug possession stealing mother assault currently incarcerated one mention expects anything except mother violation court order protection asked sends money speaks weekly well sends letter back father death seven month bout illness nearly two year father passed graduated high school left college state would send mom money college distinctly remember instance using stipend received december buy mother brother christmas gift knew would anything one reciprocated expect honestly care either became pattern next decade particular c

In [344]:
data.isnull().sum()

,0
title,0
selftext,872
created_utc,0
author,0
subreddit,0
new_title,0
new_selftext,872


In [345]:
data

,title,selftext,created_utc,author,subreddit,new_title,new_selftext
0,Is this considered domestic violence?,\nMe (28f) nd my boyfriend (34m) have been in ...,1.720186e+09,Double-Back-777,relationships,Is considered domestic violence?,f nd boyfriend relationship year got argument ...
1,My family expects me (30F) to leave my family ...,My mother is disabled due to a neurological il...,1.561784e+09,LittleMissPotatoe,relationships,My family expects (30F) leave family (and smal...,mother disabled due neurological illness diagn...
2,My [22f] boyfriend [23m] got arrested on a dom...,"My boyfriend, Trevor and I live with a family ...",1.471977e+09,Upandaway3435,relationships,My [22f] boyfriend [23m] got arrested domestic...,boyfriend trevor live family member moved aunt...
3,[UPDATE] Me [28 F] with my BF [29 M] of 3 year...,Original post: https://www.reddit.com/r/relati...,1.443372e+09,annoyedthrw,relationships,"[UPDATE] Me [28 F] BF [29 M] 3 years, parent w...",original post read reply long talk bf boundary...
4,Last night my (25F) boyfriend of 6 months (30M...,"Basically the title, but I will elaborate a li...",1.690806e+09,Stock-Property1326,relationships,Last night (25F) boyfriend 6 month (30M) told ...,basically title elaborate little met jacob mon...
...,...,...,...,...,...,...,...
13242,What's the worst relationship advice you've ev...,I was told to divorce my husband for being pou...,1.445544e+09,anxiousgirl89,AskWomen,What worst relationship advice ever seen given...,told divorce husband pouty antisocial friend p...
13243,Do you believe that love is the most important...,My sister's boyfriend treats her like shit. He...,1.420999e+09,UFOtookmysheep,AskWomen,Do believe love important thing relationship?,sister boyfriend treat like shit call woman ex...
13244,Why do so many women love stories such as Twil...,I find it perplexing that so many women enjoy ...,1.413976e+09,Bendersass,AskWomen,Why many woman love story Twilight 50 shade gr...,find perplexing many woman enjoy story fantasi...
13245,What are some small things that you silently j...,NaN,1.544510e+09,MrRomantico,AskWomen,What small thing silently judge prospective ro...,NaN


In [346]:
columns_to_drop = ['title', 'selftext', 'created_utc', 'author', 'subreddit']
# Garder uniquement les colonnes existantes dans le DataFrame
columns_to_drop = [col for col in columns_to_drop if col in data.columns]

data = data.drop(columns=columns_to_drop)
data

,new_title,new_selftext
0,Is considered domestic violence?,f nd boyfriend relationship year got argument ...
1,My family expects (30F) leave family (and smal...,mother disabled due neurological illness diagn...
2,My [22f] boyfriend [23m] got arrested domestic...,boyfriend trevor live family member moved aunt...
3,"[UPDATE] Me [28 F] BF [29 M] 3 years, parent w...",original post read reply long talk bf boundary...
4,Last night (25F) boyfriend 6 month (30M) told ...,basically title elaborate little met jacob mon...
...,...,...
13242,What worst relationship advice ever seen given...,told divorce husband pouty antisocial friend p...
13243,Do believe love important thing relationship?,sister boyfriend treat like shit call woman ex...
13244,Why many woman love story Twilight 50 shade gr...,find perplexing many woman enjoy story fantasi...
13245,What small thing silently judge prospective ro...,NaN


### <font color='#00d2d3'> Vectorisation

In [348]:
# Importer les packages
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [350]:
# Nombre de mots
voc_size = len(set(" ".join(data.new_title).split()))
# voc_size2 = len(set(" ".join(data.new_selftext).split()))
# Initialiser le modèle Tokenizer
tokenizer = Tokenizer(num_words=voc_size, split=' ')
# Entrainer les données
tokenizer.fit_on_texts(data['new_title'].values)
# Vectoriser
X = tokenizer.texts_to_sequences(data['new_title'].values)
# Padding
X = pad_sequences(X)

### <font color='#00d2d3'> Sentiment Analysis avec Bi-directional LSTM

In [351]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Dropout

In [352]:
# Dimension des vecteurs
emb_dim=100
model=Sequential()
model.add(Embedding(voc_size, emb_dim, input_length=X.shape[1]))
model.add(Bidirectional(LSTM(200, return_sequences=True)))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(200)))
model.add(Dropout(0.4))
model.add(Dense(1,activation='sigmoid'))
# Compiler le modèle
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# Afficher le summary
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
